In [ ]:
import keras
batch_size = 32
input_path = "D:\Tesi Dataset\Z_Test_Dataset\\256_resized_Face"
output_path= "D:\Tesi Dataset\Z_Test_Dataset\\256_resized_UV"

training_size = 0.7
random_seed=13
steps_per_epoch=None
steps_per_validation=None
epochs=50

## Data Preprocessing


In [ ]:
import os
input_img_paths = sorted(
    [
        os.path.join(input_path, fname)
        for fname in os.listdir(input_path)
        if fname.endswith(".png")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(output_path, fname)
        for fname in os.listdir(output_path)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

if len(input_img_paths) != len(target_img_paths):
    raise ValueError("The number of images doesn't match.")
print("Number of samples:", len(input_img_paths))

In [ ]:
from tensorflow import data as tf_data
from tensorflow import io as tf_io
from tensorflow import image as tf_image

def get_dataset(
    batch_size,
    input_img_paths,
    target_img_paths,
    max_dataset_len=None,
):
    """Returns a TF Dataset."""
    def load_img(input_img_path, target_img_path):
        input_img = tf_io.read_file(input_img_path)
        input_img = tf_io.decode_png(input_img, channels=3)
        input_img = tf_image.convert_image_dtype(input_img, "float32")

        target_img = tf_io.read_file(target_img_path)
        target_img = tf_io.decode_png(target_img, channels=3)
        target_img = tf_image.convert_image_dtype(target_img, "float32")

        return input_img, target_img

    if max_dataset_len:
        input_img_paths = input_img_paths[:max_dataset_len]
        target_img_paths = target_img_paths[:max_dataset_len]
    dataset = tf_data.Dataset.from_tensor_slices((input_img_paths, target_img_paths))
    dataset = dataset.map(load_img, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.batch(batch_size)

In [ ]:
import random

# Split our img paths into a training and a validation set
random.Random(random_seed).shuffle(input_img_paths)
random.Random(random_seed).shuffle(target_img_paths)

K=int(len(input_img_paths)*training_size/batch_size)*batch_size
T=int(len(input_img_paths)*(training_size+(1-training_size)/2)/batch_size)*batch_size

train_input_img_paths = input_img_paths[:K]
train_target_img_paths = target_img_paths[:K]

val_input_img_paths = input_img_paths[K:T]
val_target_img_paths = target_img_paths[K:T]

test_input_img_paths = input_img_paths[T:]
test_target_img_paths = target_img_paths[T:]

train_dataset = get_dataset(
    batch_size,
    train_input_img_paths,
    train_target_img_paths,
)
valid_dataset = get_dataset(
    batch_size, val_input_img_paths, val_target_img_paths
)
test_dataset = get_dataset(
    1, test_input_img_paths, test_target_img_paths
)

print("Size Train Set: ",len(train_dataset),"\nSize Valid Set: ",len(valid_dataset),"\nSize Test Set: ",len(test_dataset))

# train_dataset=train_dataset.repeat()
# valid_dataset=valid_dataset.repeat()
# test_dataset=test_dataset.repeat()

## Model


In [ ]:
from keras.layers import  Conv2D, Conv2DTranspose, SpatialDropout2D
model = keras.Sequential()

def encoder_block(model, num_filters, depth=3):
    model.add(Conv2D(num_filters, (3, 3),strides=2, activation='relu', padding='same',name=f"{num_filters}_Conv_1"))
    
    for i in range(depth-1):
        model.add(Conv2D(num_filters, (3, 3), activation='relu', padding='same',name=f"{num_filters}_Conv_{i+2}"))
    
    return model

def decoder_block(model, num_filters, depth=3):
    model.add(Conv2DTranspose(num_filters, (3, 3),strides=2, activation='relu', padding='same',name=f"{num_filters}_T-Conv_1"))

    for i in range(depth-1):
        model.add(Conv2DTranspose(num_filters, (3, 3), activation='relu', padding='same',name=f"{num_filters}_T-Conv_{i+2}"))

    return model

def autoencoder():
    # Encoder
    model.add(Conv2D(3, (3, 3), activation='relu', padding='same', input_shape=(256, 256,3),name="input"))
    
    encoder_block(model, 32,depth=3)
    encoder_block(model, 64,depth=5)
    encoder_block(model, 128,depth=7)
    encoder_block(model, 256,depth=9)

    # Decoder
    decoder_block(model, 256,depth=9)
    decoder_block(model, 128,depth=7)
    decoder_block(model, 64,depth=5)
    decoder_block(model, 32,depth=3)

    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same',name="output"))

    # Compile the model
    model.compile(optimizer='adamax', loss='binary_crossentropy',
                  metrics=[keras.metrics.MeanAbsoluteError()])
    
    return model

model = autoencoder()

# Summarize the model
model.summary()
keras.utils.plot_model(model, to_file="autoencoder.png", show_shapes=True)

## Training


In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=15),
    keras.callbacks.ModelCheckpoint("checkpoint/Interim_Model.keras", save_best_only=True),
    keras.callbacks.CSVLogger("checkpoint/Logger"),
]

model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=valid_dataset,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_steps=steps_per_validation,
    callbacks=callbacks,
)

## Testing


In [ ]:
model.evaluate(test_dataset)

#### Predictions Visualization


In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
from src.visualization.image_plot import image_plot
from itertools import islice
import numpy as np  

for j,k in islice(zip(test_dataset, predictions), 10):
    image_plot(np.concatenate((j[1].numpy()[0], k), axis=1))

## Model Save


In [ ]:
model.save("Model.keras")